#  Resources Used

- https://www.youtube.com/watch?v=lV09_8432VA - Optimizing with TensorBoard - Deep Learning w/ Python, TensorFlow & Keras p.5

# Imports

In [13]:
# -------------- Modelling Packages --------------
# For modeling
from keras.models import Model
from keras.layers import Concatenate, Input, Dense
from keras.layers.recurrent import LSTM

# Callback Functions
from keras.callbacks import TensorBoard, ModelCheckpoint

# For Timestamping Models
import time

# -------------- General Packages --------------
# Data Manipulation
import pandas as pd
import numpy as np

# For Saving Files
import pickle
import os

# Loading The Dataset

In [16]:
# Given the split dataset directory, return the train/test split
def load_dataset(split_data_dir):
    pickle_in = open(split_data_dir+'X_train.pickle','rb')
    X_train = pickle.load(pickle_in)
    
    pickle_in = open(split_data_dir+'X_test.pickle','rb')
    X_test = pickle.load(pickle_in)

    pickle_in = open(split_data_dir+'y_train.pickle','rb')
    y_train = pickle.load(pickle_in)

    pickle_in = open(split_data_dir+'y_test.pickle','rb')
    y_test = pickle.load(pickle_in)
    
    return X_train,X_test,y_train,y_test

def load_tokenizer(tokenizer_dir):
    pickle_in = open(tokenizer_dir,'rb')
    t = pickle.load(pickle_in)
    return t
    
split_data_dir = './split_data/'
X_train,X_test,y_train,y_test = load_dataset(split_data_dir)

tokenizer_dir = 'tokenizer.pickle'
t = load_tokenizer(tokenizer_dir)

# Parameters

In [17]:
# -------------- Tokenizer Values --------------
SENTENCE_SIZE = 20

# -------------- Layer Size Parameters --------------
LSTM_SIZE = 50
DENSE_SIZE = 20


# -------------- DIRECTORIES --------------
MODEL_DIR = './models/'
NAME = 'FN-{}S-{}LSTM-{}D-{}.hdf5'.format(SENTENCE_SIZE,LSTM_SIZE,DENSE_SIZE,time.time())
log_dir = os.path.join("logs",NAME)


# -------------- Compile Parameters --------------
activation = 'softmax'
optimizer = 'RMSProp'
loss = 'sparse_categorical_crossentropy'
metrics = ['accuracy']

# -------------- Callbacks --------------
# access tensorboard from the command line: tensorboard --logdir logs/
tensorboard = TensorBoard(log_dir=log_dir) 
checkpointer = ModelCheckpoint(MODEL_DIR+NAME, 
                               monitor='val_accuracy', 
                               verbose=1, 
                               save_best_only=True, 
                               mode='auto')
callbacks=[tensorboard,checkpointer]


# -------------- Fitting Parameters --------------
epochs = 100
batch_size = 128

# Model Structure

In [18]:
# FIRST MODEL: TITLE1_EN
first_input = Input((SENTENCE_SIZE,1))
first_LSTM = LSTM(LSTM_SIZE)(first_input)

# SECOND MODEL: TITLE2_EN
second_input = Input((SENTENCE_SIZE,1))
second_LSTM = LSTM(LSTM_SIZE)(second_input)

# MERGE MODEL
merged = Concatenate(axis=1)([first_LSTM, second_LSTM])
merged_dense = Dense(DENSE_SIZE)(merged)
output_layer = Dense(3, activation='softmax')(merged_dense)

model = Model(inputs=[first_input, second_input], outputs=output_layer)
model.compile(optimizer=optimizer, loss=loss,metrics=metrics)

# Training

In [12]:
os.makedirs(os.path.dirname(MODEL_DIR), exist_ok=True)

# Training the model
model.fit([X_train[:,:SENTENCE_SIZE], X_train[:,SENTENCE_SIZE:]], y_train,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=([X_test[:,:SENTENCE_SIZE], X_test[:,SENTENCE_SIZE:]], y_test),
          callbacks=callbacks)

Train on 14667 samples, validate on 5052 samples
Epoch 1/100
14667/14667 [==============================] - 10s 680us/step - loss: 1.0699 - accuracy: 0.4096 - val_loss: 1.0657 - val_accuracy: 0.4058

Epoch 00001: val_accuracy did not improve from 0.50297
Epoch 2/100
14667/14667 [==============================] - 9s 605us/step - loss: 1.0477 - accuracy: 0.4339 - val_loss: 1.0490 - val_accuracy: 0.4192

Epoch 00002: val_accuracy did not improve from 0.50297
Epoch 3/100
14667/14667 [==============================] - 9s 628us/step - loss: 1.0405 - accuracy: 0.4423 - val_loss: 1.0357 - val_accuracy: 0.4406

Epoch 00003: val_accuracy did not improve from 0.50297
Epoch 4/100
14667/14667 [==============================] - 9s 636us/step - loss: 1.0339 - accuracy: 0.4481 - val_loss: 1.0318 - val_accuracy: 0.4507

Epoch 00004: val_accuracy did not improve from 0.50297
Epoch 5/100
14667/14667 [==============================] - 10s 669us/step - loss: 1.0284 - accuracy: 0.4494 - val_loss: 1.0182 - v

KeyboardInterrupt: 

# Predicting

In [ ]:
#prediction = model.predict([X.iloc[0][:n].reshape(1,20,1),X[.loc[0][:n].reshape(1,20,1)])